In [ ]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd

In [ ]:
folder_mask = r"C:\Segment\Test\mask"
folder_preds = r"C:\Segment\Test\predict"
out_file = r"C:\Segment\Test\metrics.xls"

In [ ]:
class CalculateMetrics:
    def __init__(self, folder_mask, folder_preds, out_file):
        self.folder_mask = folder_mask
        self.folder_preds = folder_preds
        self.out_file = out_file
        # calculate results
        self.pairs = self.GetPairs(self.folder_mask, self.folder_preds)
        self.metrics_list = self.GetMetricsList(self.pairs)
        self.GetResultStats(self.metrics_list, self.out_file)
        
    def GetPairs(self, folder_mask, folder_preds):    
        files_mask = [os.path.join(folder_mask, file) for file in os.listdir(folder_mask) if file.endswith('.shp')]
        files_preds= [os.path.join(folder_preds, file) for file in os.listdir(folder_preds) if file.endswith('.shp')]
        pairs = []
        for file_mask in files_mask:
            for file_preds in files_preds:
                if os.path.basename(file_preds) == os.path.basename(file_mask):
                    pairs.append([file_mask, file_preds])
        return pairs

    def GetMetrics(self, mask_df, pred_df):
        # count areas
        inter_df = mask_df.overlay(pred_df, how='intersection')
        tp_area = sum(inter_df.area)
        fn_area = sum(mask_df.area) - sum(inter_df.area)
        fp_area = sum(pred_df.area) - sum(inter_df.area)
        # count metrics
        prec = tp_area/(tp_area+fp_area)
        recall = tp_area/(tp_area+fn_area)
        f1 = (2*prec*recall)/(prec+recall)
        return [prec, recall, f1]

    def GetMetricsList(self, pairs):
        metrics_list = []
        for pair in pairs:
            mask = pair[0]
            pred = pair[1]
            mask_df = gpd.read_file(mask)
            pred_df = gpd.read_file(pred)
            metrics = self.GetMetrics(mask_df, pred_df)
            metrics_list.append(metrics)
        return metrics_list

    def GetResultStats(self, metrics_list, out_file):
        data = {'prec': [metrics[0] for metrics in metrics_list],
               'recall': [metrics[1] for metrics in metrics_list],
               'f1': [metrics[2] for metrics in metrics_list]}
        df = pd.DataFrame(data)
        prec_res = np.mean(df.prec.tolist())
        recall_res = np.mean(df.recall.tolist())
        f1_res = np.mean(df.f1.tolist())
        df.loc[len(df.index)] = [prec_res, recall_res, f1_res] 
        df.to_excel(out_file)
        
CalculateMetrics(folder_mask, folder_preds, out_file)